In [1]:
import torch
device = torch.device("dml")
x = torch.ones(3, 3, device=device)
print(x)


RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, maia, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: dml

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from timm import create_model
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score
from PIL import Image


# Import Libraries & Set Configuration

In [7]:
import torch
device = torch.device("cpu")
print("Using CPU for training. It will be slow but will work.")


# Paths
DATA_DIR = "Dataset_patches_small"  # Folder containing 'tumor' and 'normal' subfolders
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001
IMG_SIZE = 256
NUM_CLASSES = 2  # Tumor vs Normal


Using CPU for training. It will be slow but will work.


# Define Image Transformations

In [8]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load & Split Dataset

In [9]:
# Load full dataset
full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=train_transform)

# Split dataset (80% Train, 10% Val, 10% Test)
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Check dataset split
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


Train: 80, Val: 10, Test: 10


# Load Pretrained EfficientNet-B0

In [10]:
# Load EfficientNet-B0 with pretrained weights
model = create_model("efficientnet_b0", pretrained=True, num_classes=NUM_CLASSES)
model.to(device)

# Define Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)


# Training & Validation Function

In [11]:
def train(model, train_loader, val_loader, criterion, optimizer, epochs):
    best_acc = 0.0  # Track best validation accuracy

    for epoch in range(epochs):
        # Training Phase
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        
        train_acc = correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}")

        # Validation Phase
        model.eval()
        val_correct, val_total = 0, 0
        val_preds, val_labels = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, dim=1)

                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        val_auc = roc_auc_score(val_labels, val_preds)

        print(f"Validation Acc: {val_acc:.4f}, AUC: {val_auc:.4f}")

        # Save Best Model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("Model saved!")

train(model, train_loader, val_loader, criterion, optimizer, EPOCHS)


Epoch [1/10], Loss: 2.7916, Train Acc: 0.7250
Validation Acc: 1.0000, AUC: 1.0000
Model saved!
Epoch [2/10], Loss: 0.0462, Train Acc: 0.9875
Validation Acc: 1.0000, AUC: 1.0000
Epoch [3/10], Loss: 0.5160, Train Acc: 0.9500
Validation Acc: 1.0000, AUC: 1.0000
Epoch [4/10], Loss: 0.5848, Train Acc: 0.9000
Validation Acc: 0.9000, AUC: 0.9000
Epoch [5/10], Loss: 0.2690, Train Acc: 0.9750
Validation Acc: 1.0000, AUC: 1.0000
Epoch [6/10], Loss: 0.1236, Train Acc: 0.9500
Validation Acc: 1.0000, AUC: 1.0000
Epoch [7/10], Loss: 0.0544, Train Acc: 0.9875
Validation Acc: 1.0000, AUC: 1.0000
Epoch [8/10], Loss: 0.0616, Train Acc: 0.9875
Validation Acc: 1.0000, AUC: 1.0000
Epoch [9/10], Loss: 0.0417, Train Acc: 0.9875
Validation Acc: 1.0000, AUC: 1.0000
Epoch [10/10], Loss: 0.0014, Train Acc: 1.0000
Validation Acc: 0.9000, AUC: 0.9000


# Model Evaluation on Test Set

In [12]:
def evaluate(model, test_loader):
    model.eval()
    test_correct, test_total = 0, 0
    test_preds, test_labels = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = test_correct / test_total
    test_auc = roc_auc_score(test_labels, test_preds)

    print(f"Test Accuracy: {test_acc:.4f}, Test AUC: {test_auc:.4f}")

# Load best model and evaluate
model.load_state_dict(torch.load("best_model.pth"))
evaluate(model, test_loader)


C:\Users\MANGAL\AppData\Local\Temp\ipykernel_9508\1349968558.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Test Accuracy: 1.0000, Test AUC: 1.0000


# Model Inference (Predict on a New Image)

In [32]:
def predict(image_path):
    image = Image.open(image_path)
    image = val_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        pred = torch.argmax(output).item()

    return "Tumor" if pred == 0 else "Normal"

print(predict(file_path))

'''
folder_path = "./Dataset_patches/tumor"
# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    print(f"{filename}: {predict(file_path)}")
    '''


TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_0.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_10.png: Tumor
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_100.png: Tumor
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1000.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1001.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1002.png: Tumor
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1003.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1004.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1005.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1006.png: Normal
TCGA-A2-A3XV-01A-02-TSB.FF8434E6-B703-43FE-AC0A-AE53131F1EC6_patch_1007.png: Tumor
TCGA-

KeyboardInterrupt: 